In [3]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [4]:
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
  return (next_v, prev_d + 1)

def complete(item):
  v, old_d, new_d = item[0], item[1][0], item[1][1]
  return (v, old_d if old_d is not None else new_d)

def filter_clean(item):
    prev_vert, prev_dist, next_vert = item[0], item[1][0], item[1][1]
    return (prev_vert, prev_dist),(next_vert, prev_dist + 1)

n = 400  # number of partitions

In [5]:
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()

In [6]:
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()


In [7]:
x = 12
d = 0
distances = sc.parallelize([(x, d)]).partitionBy(n)

In [8]:
candidates = distances.join(forward_edges, n).map(step)
new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
vertex_path = new_distances.join(forward_edges, n).filter(lambda i: i[1][0] == 1)
current_full_path = vertex_path.flatMap(filter_clean)
entire_path_to_proceed = distances.fullOuterJoin(current_full_path,n).map(complete, True).cache()
current_working_vertex = new_distances.join(forward_edges, n).filter(lambda i: i[1][0] == 1).map(step)
new_vertices = current_working_vertex.join(forward_edges, n).map(step).cache()
temp_set = new_vertices.join(forward_edges, n).flatMap(filter_clean).filter(lambda i: i[0] == 52 or i[0] == 107)
entire_path_to_proceed_2 = entire_path_to_proceed.fullOuterJoin(temp_set,n).map(complete,True).cache()
current_working_vertex = new_vertices.join(forward_edges, n).map(step).cache()
current_working_vertex_2 = current_working_vertex.join(forward_edges,n).flatMap(filter_clean).filter(lambda i: i[0] == 20)
entire_path_to_proceed_3= entire_path_to_proceed_2.fullOuterJoin(current_working_vertex_2,n).map(complete,True).cache()
newest_vertex = current_working_vertex_2.filter(lambda i: i[0]==20).cache()
next_path = newest_vertex.join(forward_edges,n).map(step).filter(lambda i: i[0]==23).cache()
entire_path_to_proceed_4 = entire_path_to_proceed_3.fullOuterJoin(next_path,n).map(complete,True).cache()
next_path_3 = next_path.join(forward_edges,n).map(step).filter(lambda i: i[0]==274).cache()
entire_path_to_proceed_5 = entire_path_to_proceed_4.fullOuterJoin(next_path_3,n).map(complete,True).cache()
entire_path_to_proceed_5 = entire_path_to_proceed_4.fullOuterJoin(next_path_3,n).map(complete,True).cache()
next_path_4 = next_path_3.join(forward_edges,n).map(step).filter(lambda i: i[0]==34).cache()
entire_path_to_proceed_6 = entire_path_to_proceed_5.fullOuterJoin(next_path_4,n).map(complete,True).cache()
final_RDD = entire_path_to_proceed_6.sortBy(lambda i: i[1]).cache()
final_RDD.map(lambda i:i[0]).collect()

[12, 422, 53, 52, 107, 20, 23, 274, 34]

In [13]:
answer = final_RDD.take(9)
answer_list = []
for i in answer:
    answer_list.append(i[0])
print(answer_list)

[12, 422, 53, 52, 107, 20, 23, 274, 34]


In [21]:
no_space = str(answer_list)
no_space = no_space.replace(" ","")
no_space = no_space.replace('[','')
no_space = no_space.replace(']','')
print(no_space)

12,422,53,52,107,20,23,274,34
